In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from datetime import datetime

from os.path import join

In [ ]:
ROOT_DIR = '/media/sf_JD/DP'
inpath_vector = join(ROOT_DIR, 'reference/2021/vyjezdy_2021_zonal_stats.gpkg')

In [ ]:
class Time_series:
    def __init__(self, year, relative_orbits=(22,73,95,146)):
        self.year = year
        self.sar_gdf = gpd.read_file(join(ROOT_DIR, 'reference', str(year), f'vyjezdy_{year}_zonal_stats.gpkg'))#inpath_vector)
        # will need to be split based on RO
        self.sar_keys = [key for key in self.sar_gdf.keys() if 'VV_' in key or 'VH_' in key]
        # sar dates should be changed based on relative orbit - maybe a dict of ROs with sar dates for each?
        self.sar_dates = list(set([datetime.strptime(key[7:16], '%Y_%m%d') for key in self.sar_keys]))
        # for ro in ROs:
        #     self.[ro].sar_dates = list(set([datetime.strptime(key[7:16], '%Y_%m%d') for key in self.[ro].sar_keys]))
        #     self.[ro].sar_dates.sort()
        self.sar_dates.sort()
        self.metrics = self._extract_metrics()
        
        self.obs_gdf = self.sar_gdf.filter(regex='\A\d.*\d\Z', axis='columns')
        self.obs_dates = [datetime.strptime(f'{year}_{date}', '%Y_%d_%m') for date in self.obs_gdf.keys()]
        
        
    def _extract_metrics(self):
        ros = set([i[:3] for i in self.sar_keys])
        pols = set([i[4:6] for i in self.sar_keys])
        stats = set([i[22:] for i in self.sar_keys])

        metrics = {}
        for ro in ros:
            for stat in stats:
                for pol in pols:
                    cols = [coh for coh in self.sar_keys if stat in coh and pol in coh and ro in coh]
                    cols.sort()
                    metrics[f'{stat}_{pol}_RO{ro}'] = cols
        return metrics
        
    def list_metrics(self):
        print(self.metrics.keys())
    
    def plot_series_single(self, plot_idx=0, stat='mean', pol='VH', ro=146):
        fig, ax1 = plt.subplots()

        ax1.set_xlabel('Time')
        ax1.set_ylabel('S1 Coherence', color = 'red') 
        ax1.set_ylim(0.25,0.5)
        metric = f'{stat}_{pol}_RO{ro:03}'
        y_coh = self.sar_gdf.iloc[plot_idx].loc[self.metrics[metric]]
        plot_1 = ax1.plot(self.sar_dates, y_coh, color='red') 
        ax1.tick_params(axis ='y', labelcolor = 'red')

        # Adding Twin Axes
        ax2 = ax1.twinx() 
        ax2.set_ylabel('In situ grass height [cm]', color = 'green')
        ax2.set_ylim(0,120)
        plot_2 = ax2.bar(self.obs_dates, self.obs_gdf.iloc[plot_idx], color='green')
        ax2.tick_params(axis ='y', labelcolor = 'green')

        # Show plot
        plt.show()
    
    def plot_series_all(self, stat='mean', pol='VH', ro=146):
        fig, (axs) = plt.subplots(ncols=6, nrows=10, squeeze=False, sharex='all', sharey='all')
        axs_flat = axs.flatten()
        metric = f'{stat}_{pol}_RO{ro:03}'

        for idx, row in enumerate(self.sar_gdf.iterrows()):
            x = [datetime.strptime(string[7:16], '%Y_%m%d') for string in row[1][self.metrics[metric]].keys()]
            y = row[1][self.metrics[metric]]
            axs_flat[idx].plot(x, y)

In [ ]:
hello_world = Time_series(2021)

In [ ]:
hello_world.plot_series_single(plot_idx=16, ro=146)

In [ ]:
hello_world.plot_series_all()